In [1]:
%useLatestDescriptors
%use dataframe(1.0.0-Beta3), kandy

In [2]:
import com.fasterxml.jackson.dataformat.xml.XmlMapper
import com.fasterxml.jackson.module.kotlin.registerKotlinModule

val mapper = XmlMapper()
    .registerKotlinModule()

In [4]:
import ds.types.raw.pointz.PointzAggregatorUsers
import java.io.File

val data = mapper.readValue<PointzAggregatorUsers>(File("../data/PointzAggregator-AirlinesData.xml"), PointzAggregatorUsers::class.java)

In [5]:
import ds.utils.db.DB
import ds.utils.db.executeBatch

DB.executeBatch("""
	INSERT INTO airlines.pointz_users (uid, first_name, last_name, id) VALUES (?, ?, ?, ?);
""", data.users) { user ->
    setLong(1, user.uid)
    setString(2, user.name.first)
    setString(3, user.name.last)
    setObject(4, user.id)
    addBatch()
}

In [6]:
DB.executeBatch("INSERT INTO airlines.airport (code) VALUES (?) ON CONFLICT DO NOTHING;", data.users) { user ->
    user.cards?.cards?.forEach { card ->
        card.activities?.activities?.forEach { activity ->
            setString(1, activity.departure.uppercase());
            addBatch()
            setString(1, activity.arrival.uppercase());
            addBatch()
        }
    }
}

In [7]:
import ds.utils.db.DB
import ds.utils.db.executeBatch
import java.sql.Date
import java.time.format.DateTimeFormatter

DB.executeBatch("""
	INSERT INTO airlines.pointz_activities (bonus_program, number, code, date, departure, arrival, fare, user_id)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?);
""", data.users) { user ->
    user.cards?.cards?.forEach { card ->
        card.activities?.activities?.forEach { activity ->
			setString(1, card.bonusProgram)
            setString(2, card.number)
            setString(3, activity.code)
            setDate(4, Date.valueOf(activity.date))
            setString(5, activity.departure.uppercase())
            setString(6, activity.arrival.uppercase())
            setString(7, activity.fare)
            setObject(8, user.id)
            addBatch()
        }
    }
}